NYCTaxi 2013
===========

In 2014 Chris Whong successfully submitted a FOIA request to obtain the records of all taxi rides in New York City for the year of 2013.  

http://chriswhong.com/open-data/foil_nyc_taxi/

We use [dask.dataframe](dask.pydata.org/en/latest/dataframe.html) and [castra](github.com/blaze/castra) to interact with this data.

### Download and untar

This should take less than two minutes

In [ ]:
!wget  https://storage.googleapis.com/blaze-data/nyc-taxi/castra/nyc-taxi-2013.castra.tar

In [ ]:
# This takes about a minute
!tar -xf tripdata.castra.tar

### Wrap with dask.dataframe

In [ ]:
import dask.dataframe as dd
df = dd.from_castra('tripdata.castra/')
df.head()

### Set up progress bar

This lets us know how long our `dask.dataframe` computations take.

In [ ]:
from dask.diagnostics import ProgressBar
progress_bar = ProgressBar()
progress_bar.register()

## Play



### How many passengers per ride?

In [ ]:
df.passenger_count.value_counts().compute()

### How many medallions on the road per day?

In [ ]:
%matplotlib inline
df.medallion.resample('1d', how='nunique').compute().plot()

### Lets consider this per hour

We'll have to switch from matplotlib to bokeh so that we can zoom around

In [ ]:
hourly = df.medallion.resample('1h', how='nunique').compute()

In [ ]:
from bokeh.charts import TimeSeries, output_notebook, show
output_notebook()

fig = TimeSeries(hourly.values, hourly.index, title='Cabs on the road',
                 xlabel='Time', ylabel='Number of cabs')
show(fig)